In [3]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from PIL import Image
import torch
from diffusers import (
    ConsistencyDecoderVAE,
    PixArtAlphaPipeline,
    DPMSolverMultistepScheduler,
)
# from sa_solver_diffusers import SASolverScheduler
from transformers import T5EncoderModel
from utils import *

styles = {k["name"]: (k["prompt"], k["negative_prompt"]) for k in style_list}
STYLE_NAMES = list(styles.keys())
DEFAULT_STYLE_NAME = "(No style)"
SCHEDULE_NAME = ["DPM-Solver", "SA-Solver"]
DEFAULT_SCHEDULE_NAME = "DPM-Solver"
NUM_IMAGES_PER_PROMPT = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
model_id = "PixArt-alpha/PixArt-XL-2-512x512"
use_res = 512

pipe = PixArtAlphaPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            use_safetensors=True,
        )

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pipe.to(device)

result_dir = f"results/{model_id.split('/')[-1]}"
os.makedirs(result_dir, exist_ok=True)

Loading pipeline components...: 100%|██████████| 5/5 [00:06<00:00,  1.22s/it]


In [5]:
styles

{'(No style)': ('{prompt}', ''),
 'Cinematic': ('cinematic still {prompt} . emotional, harmonious, vignette, highly detailed, high budget, bokeh, cinemascope, moody, epic, gorgeous, film grain, grainy',
  'anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured'),
 'Photographic': ('cinematic photo {prompt} . 35mm photograph, film, bokeh, professional, 4k, highly detailed',
  'drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly'),
 'Anime': ('anime artwork {prompt} . anime style, key visual, vibrant, studio anime,  highly detailed',
  'photo, deformed, black and white, realism, disfigured, low contrast'),
 'Manga': ('manga style {prompt} . vibrant, high-energy, detailed, iconic, Japanese comic style',
  'ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, Western comic style'),
 'Digital Art': ('concept art {prompt} . digital artwork, illustrative, painterly, matte pai

In [6]:
text_01 = """
Title: The Little Garden
In a small village, there was a little garden. It was filled with colorful plants and fragrant flowers. The villagers loved to visit the garden to enjoy the beauty and fresh air.

After a storm, the villagers came together to fix the garden. They worked hard to repair the damaged plants and flowers.

The villagers felt proud and happy after they fixed the garden. They were grateful for their community coming together to restore the beauty of their little village.
"""

text_02 = """
Title: The Science Mystery"What is the purpose of science?" asked Ms. Johnson. "To discover new things and make our lives better," replied Alex.

Alex works hard and conducts experiments. One day, he discovers a new way to recycle plastic waste.

Alex felt excited and proud. "I can't wait to share this with everyone!" he exclaimed.
"""

text_03 = """
Title: The Elephant's Friend
In the heart of the jungle, there lived a majestic elephant named Ellie. Ellie was known for her grace and strength, but what set her apart from other elephants was her kind and gentle nature. She was not only a symbol of strength but also a symbol of compassion.

Ellie and her friends were enjoying a peaceful day in the jungle when they heard the sound of heavy footsteps. They looked around and saw a group of poachers entering their territory. Ellie's heart raced with fear as she thought about the danger they were in. She immediately called for her friends and together they hid behind a large tree, ready to defend themselves if necessary.

Ellie and her friends were ready to defend themselves against the poachers. They stood behind the tree, watching the poachers carefully. When they saw an opportunity, they quickly ran towards the poachers, using their strength and agility to chase them away. The poachers were surprised by the elephants' bravery and quickly retreated.
"""


t1 = "In a small village, a cherished garden faced storm damage; however, united efforts of the villagers restored its beauty, fostering pride and gratitude within the community."
t2 = "Ms. Johnson asks about the purpose of science; Alex, after diligent work, excitedly discovers a novel plastic recycling method, eager to share it with everyone."
t3 = "In the jungle, the compassionate elephant Ellie and her friends bravely thwart poachers, using their strength and agility to protect their territory and drive the intruders away."



text_01 = text_01.strip().split("\n\n") + [text_01, t1]
text_02 = text_02.strip().split("\n\n") + [text_02, t2]
text_03 = text_03.strip().split("\n\n") + [text_03, t3]

In [12]:
import time

schedule = "DPM-Solver"
dpms_inference_steps = 10
dpms_guidance_scale = 4.5
total_time, num = 0, 0
for prompt in text_03:
    for style in styles.keys():
        seed, randomize_seed = 0, True
        start_time = time.time()
        seed = int(randomize_seed_fn(seed, randomize_seed))
        generator = torch.Generator().manual_seed(seed)
        
        if schedule == "DPM-Solver":
            if not isinstance(pipe.scheduler, DPMSolverMultistepScheduler):
                pipe.scheduler = DPMSolverMultistepScheduler()
            num_inference_steps = dpms_inference_steps
            guidance_scale = dpms_guidance_scale
        elif schedule == "SA-Solver":
            if not isinstance(pipe.scheduler, SASolverScheduler):
                pipe.scheduler = SASolverScheduler.from_config(
                    pipe.scheduler.config,
                    algorithm_type="data_prediction",
                    tau_func=lambda t: 1 if 200 <= t <= 800 else 0,
                    predictor_order=2,
                    corrector_order=2,
                )
            num_inference_steps = sas_inference_steps
            guidance_scale = sas_guidance_scale
        else:
            raise ValueError(f"Unknown schedule: {schedule}")
        
        base_prompt, base_neg_prompt = prompt, ""
        prompt, negative_prompt = apply_style(styles, style, base_prompt, base_neg_prompt)
              
        images = pipe(
            prompt=prompt,
            width=512,
            height=512,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            generator=generator,
            num_images_per_prompt=NUM_IMAGES_PER_PROMPT,
            use_resolution_binning=True,
            output_type="pil",
        ).images
        
        end_time = time.time()  # Record the end time
        total_time += end_time - start_time  # Calculate the duration
        avg_step_duration_ms = int(
            (duration / num_inference_steps) * 1000
        )  # Calculate average step duration in milliseconds


        images[0].save(f"{result_dir}/{style}_{num+1:02}.jpg")
        num += 1

print(f"{total_time/num:.2f}")


Setting `clean_caption=True` requires the Beautiful Soup library but it was not found in your environment. You can install it with pip:
`pip install beautifulsoup4`. Please note that you may need to restart your runtime after installation.

Setting `clean_caption` to False...

Setting `clean_caption=True` requires the Beautiful Soup library but it was not found in your environment. You can install it with pip:
`pip install beautifulsoup4`. Please note that you may need to restart your runtime after installation.

Setting `clean_caption` to False...
100%|██████████| 10/10 [00:00<00:00, 34.89it/s]

Setting `clean_caption=True` requires the Beautiful Soup library but it was not found in your environment. You can install it with pip:
`pip install beautifulsoup4`. Please note that you may need to restart your runtime after installation.

Setting `clean_caption` to False...

Setting `clean_caption=True` requires the Beautiful Soup library but it was not found in your environment. You can in

0.41
